# Data Wrangling
***********
The initial data set (li_dirtydata.csv) was gathered and structured manually using Microsoft Excel. To provide context for the origin of this dataset, I have outlined the work completed outside of this repository below.

<br>

Utilized Power Query to import data tables for seasons 1-9 from Love Island Wiki.
- Imported table columns: Name, Age, Hometown, Day Entered, Status.  

<br>

Inserted a new column "Season" for each table to identify the season of each contestant appeared on.  
- Combined the data from all seasons into one table.  

<br>

Text to Columns feature to seperate data and form new columns: 
- Hometown split into Hometown and Region.
- Status split into Status and Day Dumped. 

<br>

Find and Replace feature to format and clean data:
- Removed blank spaces from cells for consistency.
- Removed non-integers from Day Entered and Day Dumped columns.
- Normalized names containing accented characters to regular English characters for compatibility with pandas dataframes.

<br>

Manually created and gathered data for following columns: 
- Height, Hair Color, Eye Color, Ethnicity, Length of Stay, Original Cast, Casa Amor Entry, Unique Partners, and Finalist.

Please refer to the [Data Dictionary](linktoDataDictionary) for detailed descriptions of each data field and access to the data sources.

**********

### Import Data

In [1]:
import pandas as pd

In [2]:
import os
cwd = os.getcwd()

In [3]:
data = pd.read_csv(cwd +'/data/li_dirtydata.csv')

In [4]:
data.head()

,Name,Season,Gender,Age,Height,Ethnicity,Hair Color,Eye Color,Hometown,Region,Day Entered,Day Dumped,Length Of Stay,Original Cast,Casa Amor Addition,Status,Unique Partners,Finalist
0,Malia Arkian,2,Female,26,66,MENAT,Brown,Brown,Wilmslow,England,7,7,0,No,Null,Removed,0,No
1,Ben Porter,1,Male,24,71,White,Brown,Blue,Wakefield,England,32,33,1,No,Null,Dumped,0,No
2,Poppy Farnan,1,Female,22,66,White,Blonde,Blue,Cheshire,England,32,33,1,No,Null,Dumped,0,No
3,Oliver Maxwell Fernandez,2,Male,25,74,White,Brown,Brown,Hertfordshire,England,30,31,1,No,Null,Dumped,0,No
4,Shannon Singh,7,Female,22,68,Asian,Brown,Brown,Fife,Scotland,1,2,1,Yes,No,Dumped,1,No


### Checking for Duplicate Values

In [5]:
duplicates = data[data.duplicated('Name', False)]

duplicates

,Name,Season,Gender,Age,Height,Ethnicity,Hair Color,Eye Color,Hometown,Region,Day Entered,Day Dumped,Length Of Stay,Original Cast,Casa Amor Addition,Status,Unique Partners,Finalist
175,Adam Collard,8,Male,26,77,White,Brown,Blue,Newcastle,England,36,56,20,No,No,Dumped,1,No
205,Adam Collard,4,Male,22,77,White,Brown,Blue,Newcastle,England,1,32,31,Yes,No,Dumped,4,No


Adam Collard appeared on the show in two seperate seasons, I will retain both of his stays in the villa for this data set. <br>
Apart from this special case, there are no duplicates in our data. 

### Missing Values 

In [6]:
#Sample of Season 1 and 2 columns, showing Casa Amor Addition as "Null"
no_casa = data[data["Casa Amor Addition"].isin(['Null'])]

no_casa.head()

,Name,Season,Gender,Age,Height,Ethnicity,Hair Color,Eye Color,Hometown,Region,Day Entered,Day Dumped,Length Of Stay,Original Cast,Casa Amor Addition,Status,Unique Partners,Finalist
0,Malia Arkian,2,Female,26,66,MENAT,Brown,Brown,Wilmslow,England,7,7,0,No,Null,Removed,0,No
1,Ben Porter,1,Male,24,71,White,Brown,Blue,Wakefield,England,32,33,1,No,Null,Dumped,0,No
2,Poppy Farnan,1,Female,22,66,White,Blonde,Blue,Cheshire,England,32,33,1,No,Null,Dumped,0,No
3,Oliver Maxwell Fernandez,2,Male,25,74,White,Brown,Brown,Hertfordshire,England,30,31,1,No,Null,Dumped,0,No
5,James Khan,2,Male,20,75,White,Brown,Blue,Teddington,England,18,20,2,No,Null,Dumped,0,No


Casa Amor was not introduced until Season 3 of the show. <br>
As shown above, Season 1 and 2 contestants have 'Null' as their "Casa Amor Addition" value. <br>
I will replace 'Null' with 'No' for consistency.

In [7]:
data['Casa Amor Addition'] = data['Casa Amor Addition'].replace('Null', 'No')

### Status Column Edge Cases

In [8]:
#Create new dataframe where Status column values are not our desired values
status_check = data[~data['Status'].isin(['1','2','3','4','Dumped'])]

status_check

,Name,Season,Gender,Age,Height,Ethnicity,Hair Color,Eye Color,Hometown,Region,Day Entered,Day Dumped,Length Of Stay,Original Cast,Casa Amor Addition,Status,Unique Partners,Finalist
0,Malia Arkian,2,Female,26,66,MENAT,Brown,Brown,Wilmslow,England,7,7,0,No,No,Removed,0,No
24,Ollie Williams,6,Male,23,70,White,Blonde,Blue,Cornwall,England,1,4,3,Yes,No,Walked,1,No
54,Liam Llewellyn,8,Male,22,71,White,Brown,Blue,Newport,Wales,1,5,4,Yes,No,Walked,1,No
99,Niall Aslam,4,Male,23,71,Asian,Black,Brown,Coventry,England,1,9,8,Yes,No,Walked,2,No
100,Sherif Lanre,5,Male,20,73,Black,Black,Brown,London,England,1,9,8,Yes,No,Removed,1,No
142,Rykard Jenkins,2,Male,25,72,Black,Brown,Brown,Kent,England,1,15,14,Yes,No,Walked,3,No
177,Zara Holland,2,Female,20,63,White,Blonde,Green,Hull,England,1,22,21,Yes,No,Walked,3,No
204,Jacques O'Neill,8,Male,23,69,White,Brown,Blue,Cumbria,England,7,37,30,No,No,Walked,2,No
219,Amy Hart,5,Female,26,66,White,Blonde,Blue,Worthing,England,1,37,36,Yes,No,Walked,2,No
222,Sophie Gradon,2,Female,30,67,White,Brown,Brown,Newcastle,England,1,39,38,Yes,No,Walked,2,No


The contestants shown above contain 'Walked' or 'Removed' in their "Status" column. <br>
'Walked' indicates the contestant decided to leave the villa, while 'Removed' indicates the candidate was cut from the show by producers. <br>
I am going to replace these values to 'Dumped' for consistency. This will ensure that contestants who did not make it to the finale are all identified by the same value in the "Status" column.

In [9]:
data['Status'] = data['Status'].replace('Walked', 'Dumped')
data['Status'] = data['Status'].replace('Removed', 'Dumped')

### Column Names

Shorten column names for ease of use going forward.

In [11]:
#Create dictionary of abbreviated column names
new_columns = {'Hair Color': 'Hair',
               'Eye Color': 'Eye',
               'Day Entered': 'Entered',
               'Day Dumped': 'Dumped',
               'Length Of Stay': 'Stay',
               'Original Cast': 'OG',
               'Casa Amor Addition': 'Casa',
               'Unique Partners': 'Couples'}

#Rename columns specified in dictionary above
data = data.rename(columns=new_columns)

In [12]:
#Export new data to csv
data.to_csv(cwd + '/data/li_cleandata.csv', index=False)

In [13]:
#Check new csv file
check = pd.read_csv(cwd + '/data/li_cleandata.csv')

check.head()

,Name,Season,Gender,Age,Height,Ethnicity,Hair,Eye,Hometown,Region,Entered,Dumped,Stay,OG,Casa,Status,Couples,Finalist
0,Malia Arkian,2,Female,26,66,MENAT,Brown,Brown,Wilmslow,England,7,7,0,No,No,Dumped,0,No
1,Ben Porter,1,Male,24,71,White,Brown,Blue,Wakefield,England,32,33,1,No,No,Dumped,0,No
2,Poppy Farnan,1,Female,22,66,White,Blonde,Blue,Cheshire,England,32,33,1,No,No,Dumped,0,No
3,Oliver Maxwell Fernandez,2,Male,25,74,White,Brown,Brown,Hertfordshire,England,30,31,1,No,No,Dumped,0,No
4,Shannon Singh,7,Female,22,68,Asian,Brown,Brown,Fife,Scotland,1,2,1,Yes,No,Dumped,1,No


The resulting csv (li_cleandata.csv) will now be analyzed in the EDA.ipynb notebook.